In [2]:
from appgeopy import *
from my_packages import *

pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 500)

In [3]:
def differ_to_ref(series, convert_to=None):
    """
    Calculate differential values relative to the first measurement in the series.

    Parameters:
    - series (pd.Series): A pandas Series containing the measurements.
    - convert_to (str or None): Unit to convert the differential values to.
                                Accepts 'milimeter', 'centimeter', or None for 'meters'.

    Returns:
    - np.ndarray: The differential values converted to the specified unit.

    Raises:
    - ValueError: If 'convert_to' is not 'milimeter', 'centimeter', or None.

    """
    # Determine the conversion multiplier based on the desired unit.
    if convert_to == "milimeter":
        multiplier = 1000
    elif convert_to == "centimeter":
        multiplier = 100
    elif convert_to is None:
        multiplier = 1  # Keep the original unit (meters).
    else:
        raise ValueError("Invalid 'convert_to' value. Must be 'milimeter', 'centimeter', or None for meters.")

    # Calculate differential values relative to the first measurement and apply conversion.
    return np.array((series - series.iloc[0]) * multiplier, dtype=np.float16)

# -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
def cdisp_from_base(input_array):
    cdisp_ref_to_base = np.nancumsum(input_array[::-1], dtype=np.float64)
    return cdisp_ref_to_base[::-1]

# -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
# Enhanced plotting function
def PlotData_by_Column(ax, df, borehole_depth):
    """
    Plot data from each column of a DataFrame against borehole depth, using smooth interpolation.

    Parameters:
    - ax (matplotlib.axes.Axes): The axis to plot on.
    - df (pd.DataFrame): DataFrame containing columns as dates and values.
    - borehole_depth (np.ndarray): Depth values for the borehole.

    Returns:
    - None
    """
    cmap = plt.get_cmap("turbo")
    norm = Normalize(vmin=df.columns.min().toordinal(), vmax=df.columns.max().toordinal())

    # Generate the date range for normalization of colors
    actual_date_range = pd.date_range(start=df.columns[0], end=df.columns[-1], freq="1D")

    # Vectorize the interpolation process
    for select_col in df.columns:
        arr_label = select_col.strftime("%Y/%m/%d")
        date_as_ordinal = select_col.toordinal()  # Normalize date for color mapping
        color = cmap(norm(date_as_ordinal))

        # Interpolate data using the spline interpolation
        select_array = df[select_col].values
        x_new, y_new = interpolate.spline_interp(x=borehole_depth, y=select_array, factor=100)

        # Plot original data points and the interpolated curve
        ax.plot(-select_array, -borehole_depth, marker="o", linestyle=" ", lw=1.5, color=color)
        ax.plot(-y_new, -x_new, linestyle="-", lw=1.5, color=color)


# -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --
from mpl_toolkits.axes_grid1.inset_locator import inset_axes


def AddColorBar(fig, ax, df):
    modified_datetime = df.columns.strftime("%Y/%m/%d")
    sm = plt.cm.ScalarMappable(cmap="turbo", norm=plt.Normalize(vmin=0, vmax=1))
    cbaxes = inset_axes(
        ax,
        width="25%",
        height="1.25%",
        loc="lower left",
        bbox_to_anchor=(0.7, 0.05, 0.1, 20),
        bbox_transform=ax.transAxes,
        borderpad=0,
    )
    cbar = fig.colorbar(sm, cax=cbaxes, orientation="vertical")
    cbar.ax.set_yticks([0, 0.2, 0.4, 0.6, 0.8, 1])
    _len = len(modified_datetime)
    datestring_list = [
        modified_datetime[X] for X in [-1, int(_len * 0.8), int(_len * 0.6), int(_len * 0.4), int(_len * 0.2), 0]
    ]
    cbar.ax.set_yticklabels(datestring_list[::-1], fontsize=16)


# -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- --

In [16]:
well_df = pd.read_excel(r"D:\1000_SCRIPTS\003_Project002\20241029_MLCW_Classify\Xiutan_NewData.xlsx", index_col=[0])
well_df.columns = [
    datetime_handle.convert_to_datetime(ele) if type(ele) == str else pd.to_datetime(ele) for ele in well_df.columns
]
well_df = well_df.fillna(method='ffill', axis=1)
well_df = well_df.fillna(method='bfill', axis=1)
well_df = well_df.T

column_diffs = well_df.diff(axis=1)
column_diffs.iloc[:, 0] = well_df.iloc[:, 0]

filled_diffs = column_diffs.fillna(axis=0, method="ffill")
filled_diffs = filled_diffs.loc["12-2014":"2021"]

diffs_ref2first = filled_diffs.apply(lambda x: differ_to_ref(x), axis=0).astype(np.float16)
diffs_ref2first_transposed = diffs_ref2first.transpose()

cdisp_ref2base = diffs_ref2first_transposed.apply(cdisp_from_base, axis="index")
cdisp_ref2base = cdisp_ref2base*100

In [17]:
cdisp_ref2base

,2014-12-05,2015-01-13,2015-02-13,2015-03-19,2015-04-20,2015-05-14,2015-06-10,2015-07-08,2015-08-20,2015-09-25,2015-10-14,2015-11-10,2015-12-15,2016-01-12,2016-02-23,2016-03-16,2016-04-12,2016-05-25,2016-06-16,2016-07-14,2016-08-09,2016-09-26,2016-10-12,2016-11-04,2016-12-12,2017-01-18,2017-02-14,2017-03-08,2017-04-20,2017-05-04,2017-06-13,2017-07-18,2017-08-03,2017-09-11,2017-10-23,2017-11-20,2017-12-19,2018-01-08,2018-02-01,2018-03-12,2018-04-09,2018-05-08,2018-06-11,2018-07-16,2018-08-14,2018-09-11,2018-10-16,2018-11-22,2018-12-04,2019-01-11,2019-02-12,2019-03-05,2019-04-11,2019-05-13,2019-06-12,2019-07-03,2019-08-06,2019-09-03,2019-10-08,2019-11-06,2019-12-11,2020-01-14,2020-02-06,2020-03-04,2020-04-21,2020-05-05,2020-06-11,2020-07-09,2020-08-04,2020-09-09,2020-10-14,2020-11-11,2020-12-11,2021-01-12,2021-02-18,2021-03-10,2021-04-07,2021-05-04,2021-06-03,2021-07-07,2021-08-03,2021-09-10,2021-10-07,2021-11-04,2021-12-02
測點,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
NO1,0.0,0.002001,-0.024007,-0.040007,-0.055015,-0.065016,-0.057011,-0.056013,-0.050011,-0.051012,-0.042012,-0.052011,-0.042011,-0.043013,-0.059012,-0.062011,-0.064018,-0.078019,-0.077018,-0.067016,-0.073016,-0.069012,-0.064013,-0.074014,-0.063013,-0.081016,-0.102010,-0.112015,-0.122018,-0.129015,-0.128008,-0.112019,-0.113014,-0.115013,-0.127010,-0.125015,-0.120010,-0.122022,-0.129025,-0.151016,-0.161007,-0.181031,-0.206024,-0.177030,-0.183999,-0.172025,-0.188025,-0.186030,-0.181027,-0.190036,-0.213020,-0.218022,-0.223000,-0.219034,-0.206032,-0.197002,-0.196004,-0.184030,-0.203004,-0.211024,-0.201024,-0.201012,-0.214025,-0.226042,-0.239023,-0.247025,-0.235023,-0.232027,-0.233997,-0.229021,-0.248015,-0.245024,-0.241030,-0.239014,-0.259027,-0.277999,-0.290022,-0.307999,-0.328019,-0.301022,-0.299020,-0.300035,-0.313005,-0.314029,-0.306002
NO2,0.0,0.002001,-0.023006,-0.040007,-0.055015,-0.065016,-0.057011,-0.056013,-0.050011,-0.051012,-0.043013,-0.052011,-0.042011,-0.043013,-0.059012,-0.062011,-0.064018,-0.078019,-0.077018,-0.067016,-0.073016,-0.069012,-0.064013,-0.074014,-0.064013,-0.082016,-0.102010,-0.112015,-0.122018,-0.129015,-0.128008,-0.110018,-0.111013,-0.112013,-0.124010,-0.122015,-0.117010,-0.118020,-0.125023,-0.148016,-0.158007,-0.178031,-0.203024,-0.174029,-0.180999,-0.168023,-0.184023,-0.182029,-0.177026,-0.186034,-0.209019,-0.214021,-0.219999,-0.216034,-0.204031,-0.194002,-0.193004,-0.181029,-0.200004,-0.208024,-0.198024,-0.198011,-0.211024,-0.223042,-0.235022,-0.244025,-0.232023,-0.229027,-0.230997,-0.226021,-0.244014,-0.241022,-0.238029,-0.235012,-0.255026,-0.273997,-0.286020,-0.303997,-0.325019,-0.297020,-0.294019,-0.296033,-0.309004,-0.311028,-0.302000
NO3,0.0,0.002001,-0.023006,-0.040007,-0.055015,-0.065016,-0.057011,-0.057014,-0.051011,-0.052012,-0.043013,-0.052011,-0.043012,-0.044013,-0.059012,-0.062011,-0.064018,-0.078019,-0.077018,-0.068016,-0.074017,-0.070012,-0.065013,-0.075014,-0.064013,-0.082016,-0.102010,-0.112015,-0.123018,-0.129015,-0.128008,-0.109017,-0.112014,-0.112013,-0.124010,-0.123015,-0.118011,-0.119020,-0.125023,-0.148016,-0.158007,-0.178031,-0.203024,-0.173029,-0.180999,-0.169024,-0.185023,-0.182029,-0.177026,-0.186034,-0.209019,-0.215021,-0.219999,-0.216034,-0.204031,-0.195003,-0.194004,-0.182030,-0.200004,-0.208024,-0.198024,-0.198011,-0.211024,-0.222041,-0.235022,-0.244025,-0.233024,-0.230027,-0.230997,-0.226021,-0.244014,-0.241022,-0.238029,-0.234012,-0.254025,-0.272997,-0.286020,-0.302997,-0.324018,-0.297020,-0.295019,-0.297033,-0.309004,-0.310028,-0.302000
NO4,0.0,0.002001,-0.023006,-0.040007,-0.056015,-0.065016,-0.058011,-0.057014,-0.051011,-0.052012,-0.044013,-0.054012,-0.043012,-0.044013,-0.060013,-0.063011,-0.065019,-0.079020,-0.078018,-0.069016,-0.075017,-0.070012,-0.065013,-0.076015,-0.065014,-0.082016,-0.103010,-0.112015,-0.123018,-0.130015,-0.129008,-0.111018,-0.113014,-0.113013,-0.125010,-0.123015,-0.118011,-0.119020,-0.125023,-0.149016,-0.159007,-0.178031,-0.203024,-0.17402

In [ ]:
fig = plt.figure(figsize=(8.3 * 2 / 3, 11.7))

ax = fig.add_subplot(111)

PlotData_by_Column(ax, df=mlcw_df, borehole_depth=depth_arr)

visualize.configure_axis(
    ax,
    xlabel="Cumulative Compaction (cm)",
    ylabel="Depth (m)",
    title=select_station,
    hide_spines=["bottom", "right"],
    major_tick_length=10,
    minor_tick_length=5,
    tick_direction="in",
    fontsize_base=14,
)

visualize.configure_ticks(ax=ax, x_major_interval=5, x_minor_interval=1, y_major_interval=50, y_minor_interval=10)

AddColorBar(fig, ax, mlcw_df)

ax.xaxis.set_ticks_position("top")
ax.xaxis.set_label_position("top")

current_ytick_labels = [label.get_text() for label in ax.get_yticklabels()]
new_ytick_labels = [ele.replace("−", "") for ele in current_ytick_labels]
ax.set_yticklabels(new_ytick_labels)

plt.show()